# Ingest PBT News Releases

In [1]:
from tabula import read_pdf

In [2]:
%run configs.ipynb

In [3]:
#parameters cell
news_file_name = 'dec 2021 news, oct 2021 prod.pdf'

In [4]:
pbt_news_DF = read_pdf(f'{raw_data_folder}/pbt_news/2021/{news_file_name}')[0]

pbt_news_DF

'pages' argument isn't specified.Will extract only from page 1 by default.


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,Underlying Properties,Net to Trust Sales,NaN,NaN
1,NaN,Volumes,Volumes,Average,Price
2,NaN,Oil Gas,Oil Gas,Oil,Gas
3,NaN,(bbls) (Mcf),(bbls) (Mcf),(per bbl),(per Mcf)
4,Current Month,NaN,NaN,NaN,NaN
5,Waddell Ranch,"133,626 369,258","100,220 276,944*",$80.18,$8.89**
6,Texas Royalties,"18,096 12,056","16,200 10,787*",$74.85,$7.46**
7,NaN,NaN,NaN,NaN,NaN
8,Prior Month,NaN,NaN,NaN,NaN
9,Waddell Ranch,"120,643 265,108","90,482 198,831*",$70.09,$6.36**


In [5]:
cols_pdf = 'Property', 'Gross Oil Volume (PBT News)', 'Net Oil Volume (PBT News)', 'Average Oil Price', 'Average Gas Price'
pbt_news_DF.columns = cols_pdf

In [6]:
pbt_news_DF.dropna(inplace = True)
pbt_news_DF

,Property,Gross Oil Volume (PBT News),Net Oil Volume (PBT News),Average Oil Price,Average Gas Price
5,Waddell Ranch,"133,626 369,258","100,220 276,944*",$80.18,$8.89**
6,Texas Royalties,"18,096 12,056","16,200 10,787*",$74.85,$7.46**
9,Waddell Ranch,"120,643 265,108","90,482 198,831*",$70.09,$6.36**
10,Texas Royalties,"17,123 10,650","15,191 9,442*",$68.94,$7.36**


### Splitting Net Oil Column to Get Gas Volume
### Applying Text Cleaning to remove commas

In [7]:
pbt_news_DF['Gross Gas Volume (PBT News)'] = pbt_news_DF['Gross Oil Volume (PBT News)'].apply(lambda x: int(x.split(' ')[1].replace(',', '')))

pbt_news_DF['Gross Oil Volume (PBT News)'] = pbt_news_DF['Gross Oil Volume (PBT News)'].apply(lambda x: int(x.split(' ')[0].replace(',', '')))

In [8]:
pbt_news_DF

,Property,Gross Oil Volume (PBT News),Net Oil Volume (PBT News),Average Oil Price,Average Gas Price,Gross Gas Volume (PBT News)
5,Waddell Ranch,133626,"100,220 276,944*",$80.18,$8.89**,369258
6,Texas Royalties,18096,"16,200 10,787*",$74.85,$7.46**,12056
9,Waddell Ranch,120643,"90,482 198,831*",$70.09,$6.36**,265108
10,Texas Royalties,17123,"15,191 9,442*",$68.94,$7.36**,10650


### Splitting Net Oil Column to Get Gas Volume
### Applying Text Cleaning to remove commas and asterisks

In [9]:
pbt_news_DF['Net Gas Volume (PBT News)'] = pbt_news_DF['Net Oil Volume (PBT News)'].apply(lambda x: int(x.split(' ')[1]\
                                                                                  .replace(',', '')\
                                                                                 .replace('*','')))

pbt_news_DF['Net Oil Volume (PBT News)'] = pbt_news_DF['Net Oil Volume (PBT News)'].apply(lambda x: int(x.split(' ')[0].replace(',', '')))

### Cleaning Prices Columns by removing asterisks and Dollar Sign

In [10]:
pbt_news_DF['Average Gas Price'] = pbt_news_DF['Average Gas Price'].apply(lambda x: x.replace('$','').replace('**',''))

pbt_news_DF['Average Oil Price'] = pbt_news_DF['Average Oil Price'].apply(lambda x: x.replace('$','').replace('**',''))

In [11]:
#filtering table to the 'current month' values
pbt_news_DF = pbt_news_DF.head(2)

In [12]:
pbt_news_DF

,Property,Gross Oil Volume (PBT News),Net Oil Volume (PBT News),Average Oil Price,Average Gas Price,Gross Gas Volume (PBT News),Net Gas Volume (PBT News)
5,Waddell Ranch,133626,100220,80.18,8.89,369258,276944
6,Texas Royalties,18096,16200,74.85,7.46,12056,10787


In [13]:
date_from_filename = news_file_name.split(',')[1]\
                                    .split('.')[0]\
                                    .strip()\
                                    .replace(' prod', '')

In [14]:
pbt_news_DF.insert(0, 'Date', date_from_filename.title())

In [15]:
pbt_news_DF

,Date,Property,Gross Oil Volume (PBT News),Net Oil Volume (PBT News),Average Oil Price,Average Gas Price,Gross Gas Volume (PBT News),Net Gas Volume (PBT News)
5,Oct 2021,Waddell Ranch,133626,100220,80.18,8.89,369258,276944
6,Oct 2021,Texas Royalties,18096,16200,74.85,7.46,12056,10787


# Write Table to File

In [16]:
to_file_name = news_file_name.split('.')[0] + '.csv'

to_file_name

'dec 2021 news, oct 2021 prod.csv'

In [17]:
pbt_news_DF.to_csv(f'{processed_data_folder}/pbt_news_prod/{to_file_name}', index = False)